<table style="width:100%" border="0">
    <col width="20%">
    <col width="60%">
    <col width="20%">
    <tr></tr>
    <tr>
        <th style="text-align:left"><img src="Images/UNAM_logo.jpg" width="150px"></th>
        <th style="text-align:center"><span style="font-size:220%">Física Computacional</span><br/><br/><span style="font-size:150%">Tarea 10</span></th> 
        <th style="text-align:right"><img src="Images/LogoCiencias.jpg" width="170px"></th>
    </tr>
</table>

In [1]:
using SymPy

En este notebook se tiene un algoritmo para calcular automáticamente las diferencias finitas simétricas de $\:f(x)$ a ordenes más altos. Recordemos que si $\delta_{h}:\:f(x)\mapsto\:f(x+h)-f(x-h)$ es el operador derivada finita simétrica, entonces:

$$\left.\frac{d}{d\xi}\right|_{\substack{\xi=x}}=\frac{1}{h}\:\mathrm{senh}^{-1}\left(\frac{\delta_{h}}{2}\right)$$

In [2]:
function coeff_Taylor_Arcsinh(i)
    x=Sym("x");
    h=Sym("h");
    N(subs(diff(asinh(x),x,i)/factorial(Float64(i)),x=>0))  # En SymPy la función arcsenh es asinh.
end;

function coeff_Taylor_Arcsinh_rational(i)
    x=Sym("x");
    h=Sym("h");
    subs(diff(asinh(x),x,i)/factorial(i),x=>0)
end;

function finite_difference_symmetric(f)
    return (x,h)->f(x+h,h)-f(x-h,h)      # ------------------ Diferencia finita simétrica.
end;

Observemos que la función `finite_difference_symmetric` si nos de el resultado que queremos:

In [3]:
g=SymFunction("g");
rr(x,h)=g(x);
gg=finite_difference_symmetric(rr);
gg(Sym("x"),Sym("h"))

-g(-h + x) + g(h + x)

Y la segunda aplicación de `finite_difference_symmetric`:

In [4]:
gg=finite_difference_symmetric(gg);
gg(Sym("x"),Sym("h"))

-2*g(x) + g(-2*h + x) + g(2*h + x)

In [5]:
function Derivative_order(f,x,h,order)
    g(x,h) = f(x)
    der = 0
    for i in range(1,order)
        g = finite_difference_symmetric(g)
        der += coeff_Taylor_Arcsinh(i)*g(x,h)/2^i   # La división por 2^i es por que la derivada exacta es
    end                                             # proporcional a arcsenh(delta/2)
    der/h
end;
function Derivative_order_rational_coefficients(f,x,h,order)
    g(x,h) = f(x)
    der = 0
    for i in range(1,order)
        g = finite_difference_symmetric(g)
        der += coeff_Taylor_Arcsinh_rational(i)*g(x,h)/2^i
    end
    der/h
end;

Podemos probar la rutina para obtener la derivada numérica de la función $\cos(x)$ en $x=10$, sabiendo que la derivada exacta es $\:-\mathrm{sen}(x)$:

In [6]:
Derivative_order(cos,10.0,0.01,5)

0.5440211108893371

In [7]:
-sin(10.0)

0.5440211108893698

Se observa que la derivada numérica de orden 5 es igual a la exacta en ¡14 cifras significativas!. Para conocer como escala el error, expandimos la derivada numérica en serie de Taylor, por ejemplo la derivada numérica de orden 5:

In [8]:
approx=Derivative_order_rational_coefficients(SymFunction("f"),Sym("x"),Sym("h"),5)

  3*f(-5*h + x)   25*f(-3*h + x)   75*f(-h + x)   75*f(h + x)   25*f(3*h + x) 
- ------------- + -------------- - ------------ + ----------- - ------------- 
       1280            768             128            128            768      
------------------------------------------------------------------------------
                                             h                                

  3*f(5*h + x)
+ ------------
      1280    
--------------
              

In [9]:
series(approx,Sym("h"),0,8)

                               /   7           \|              
                             6 |  d            ||              
                          5*h *|------(f(xi_1))||              
                               |     7         ||              
/  d           \|              \dxi_1          /|xi_1=x    / 8\
|-----(f(xi_1))||       + ----------------------------- + O\h /
\dxi_1         /|xi_1=x                112                     

Se tiene entonces que para la derivada numérica simetrizada de orden 5 el error escala como $h^{6}$.

<div align="right" style="text-indent:0.5cm">**Alonso Torres Edgar Giovanni**</div>